# Calibrate with particles 

The idea is to run PyPTV as usual, and check the box "Use only 4 frames". The result will be in the /res folder with only quadruplets as 3D and the respective indices of 2D targets per image

If we read this dataset into the proper format, we can now reproject every 3D point in rt_is back into the image and then optimize calibration with disparity between the position of the target as detected and the reprojected center. 

In [1]:
# -*- coding: utf-8 -*-
# copy of https://github.com/alexlib/pbi/blob/master/ptv/shake.py
"""
BOOM shake shake shake the room!!!

Fine-tune calibration using the "shaking" method of comparing 3D positions 
obtained with existing calibration to their 2D projections. It's a kind of a 
feedback step over the normal calibration with known points.

Created on Sun Jan 31 13:42:18 2016

@author: Yosef Meller
"""
import argparse, yaml, numpy as np

from optv.orientation import full_calibration
from optv.parameters import ControlParams

from optv.tracking_framebuf import TargetArray, Frame
from optv.calibration import Calibration


import os
from pathlib import Path

present_folder = Path.cwd()

working_folder = Path("/home/user/Documents/repos/test_cavity")
par_path = working_folder / "parameters"
working_folder.exists(), par_path.exists()

from pyptv.ptv import py_start_proc_c

# we work inside the working folder, all the other paths are relative to this 
num_cams = 4
os.chdir(working_folder)
cpar, spar, vpar, track_par, tpar, calibs, epar = py_start_proc_c(num_cams)
assert cpar.get_num_cams() == num_cams

cpar.get_img_base_name(0).decode()


'img_3/cam1.0'

In [2]:

targ_files = [spar.get_img_base_name(c).decode().split('%d')[0].encode() for c in \
    range(num_cams)]

print(targ_files)


[b'img_3/cam1.', b'img_3/cam2.', b'img_3/cam3.', b'img_3/cam4.']


In [3]:
def backup_ori_files(cpar):
    """ backup ORI/ADDPAR files to the backup_cal directory """
    import shutil 
    for i_cam in range(cpar.get_num_cams()):
        f = cpar.get_cal_img_base_name(i_cam).decode()
        print(f"Backing up {f}.ori")
        shutil.copyfile(f + ".ori", f + ".ori.bck")
        shutil.copyfile(f + ".addpar", f + ".addpar.bck")


# Backup is the first thing to do 
backup_ori_files(cpar)

Backing up cal/cam1.tif.ori
Backing up cal/cam2.tif.ori
Backing up cal/cam3.tif.ori
Backing up cal/cam4.tif.ori


In [4]:
# Iterate over frames, loading the big lists of 3D positions and 
# respective detections.
all_known = []
all_detected = [[] for c in range(cpar.get_num_cams())]

for frm_num in range(spar.get_first(), spar.get_last() + 1): # all frames for now, think of skipping some
    frame = Frame(cpar.get_num_cams(), 
        corres_file_base = ('res/rt_is').encode(), 
        linkage_file_base= ('res/ptv_is').encode(),
        target_file_base = targ_files, 
        frame_num = frm_num)
        
    all_known.append(frame.positions())
    for cam in range(cpar.get_num_cams()):
        all_detected[cam].append(frame.target_positions_for_camera(cam))

# Make into the format needed for full_calibration.
all_known = np.vstack(all_known)

# Calibrate each camera accordingly.
for cam in range(cpar.get_num_cams()):
    detects = np.vstack(all_detected[cam])
    assert detects.shape[0] == all_known.shape[0]
    
    have_targets = ~np.isnan(detects[:,0])
    used_detects = detects[have_targets,:]
    used_known = all_known[have_targets,:]
    
    targs = TargetArray(len(used_detects))
    
    for tix in range(len(used_detects)):
        targ = targs[tix]
        targ.set_pnr(tix)
        targ.set_pos(used_detects[tix])
    
    residuals = full_calibration(calibs[cam], used_known, targs, cpar)
    
    if True:
        print(("Camera %d" % (cam + 1)))
        print((calibs[cam].get_pos()))
        print((calibs[cam].get_angles()))
    else:
        if args.output is None:
            ori = cal_args[cam]['ori_file']
            distort = cal_args[cam]['addpar_file']
        else:
            ori = args.output % (cam + 1) + '.ori'
            distort = args.output % (cam + 1) + '.addpar'
        
        calibs[cam].write(ori.encode(), distort.encode())

Can't open ascii file: img_3/cam1._targets
Camera 1
[ 117.92618753  -96.22618625 -661.83355869]
[-56.69999735   2.92738529  56.50535602]


ValueError: Orientation iteration failed, need better setup.